<a href="https://colab.research.google.com/github/KrishnaSChavan/MachineLearning/blob/main/Transfer_learning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifiying multiple images with transfer learning

In [1]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_r = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_r.extractall()
zip_r.close()

--2023-04-09 10:45:37--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 172.217.218.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  42.0MB/s    in 4.7s    

2023-04-09 10:45:41 (34.3 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [9]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import pathlib


In [5]:
import os
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"): 
  print(f"There are {dirnames} directories and {len(filenames)} images in '{dirpath}'.")

There are ['train', 'test'] directories and 0 images in '10_food_classes_10_percent'.
There are ['pizza', 'steak', 'chicken_wings', 'chicken_curry', 'sushi', 'hamburger', 'grilled_salmon', 'fried_rice', 'ramen', 'ice_cream'] directories and 0 images in '10_food_classes_10_percent/train'.
There are [] directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are [] directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are [] directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are [] directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are [] directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are [] directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are [] directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are [] directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There

In [6]:
IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = '/content/10_food_classes_10_percent/train'
test_dir = '/content/10_food_classes_10_percent/test'


train_datagen = ImageDataGenerator(rescale=(1/255.))
test_datagen = ImageDataGenerator(rescale=(1/255.))

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size = IMAGE_SHAPE,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)
test_data = train_datagen.flow_from_directory(
    test_dir,
    target_size = IMAGE_SHAPE,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


CALLBACKS

tensorboard callback




In [8]:
import datetime

def create_tensorboard_callback(dir_name,experiment_name):
    log_dir = dir_name+'/'+experiment_name+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"saving tensorboard file to {log_dir}")
    return tensorboard_callback

https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1

In [11]:
effnet_url = 'https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1'
resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'

In [15]:
def create_model(model_url,num_classes=10):

    # download model
    feature_extraction_layer = hub.KerasLayer(model_url,trainable=False,name="feature_extraction",input_shape=IMAGE_SHAPE+(3,))

    # create our own model
    model = Sequential({
        feature_extraction_layer,
        Dense(num_classes,activation='softmax',name='output')
    })

    return model


In [ ]:
# create Resnet model from tf_hub 

resnet_model = create_model(resnet_url)